[View on GitHub](https://github.com/csc-training/sscc-cp2k)

# Part 2: Vibrational analysis of the transition state

The transition state (TS) is a first-order saddle-point on the multi-dimensional potential energy surface (PES). In other words, it is a maximum along the reaction path and a minimum in all other directions. This means that a vibrational analysis of the TS should yield exactly one imaginary vibrational mode. This technique is frequently used to verify that a TS was really found.

## Step 2.1: Prepare the input file and submit the calculation

Copy the optimized TS structure and wave function restart file from the current working directory:

In [5]:
%%bash
cp ../Part-1-NEB/enol-neb-BAND5-RESTART.wfn .
cp ../Part-1-NEB/enol-neb-pos-Replica_nr_5-1.xyz .

Inspect the input file `nma.inp` and replace all `TODO`:

In [6]:
%%writefile nma.inp
&GLOBAL
  PROJECT enol-nma                      ! Project name, gets prepended to all output files
  RUN_TYPE NORMAL_MODES                 ! Type of calculation
  PRINT_LEVEL LOW                       ! Low verbosity
&END GLOBAL

&FORCE_EVAL                             ! Include DFT and system settings
  &DFT
    RESTART_FILE_NAME enol-neb-BAND5-RESTART.wfn
    BASIS_SET_FILE_NAME BASIS_MOLOPT
    &QS
      EPS_DEFAULT 1.0E-12               ! Sets convergence criteria so that energy should be accurate up to this value
    &END QS
    &POISSON                            ! How to deal with electrostatics
      PERIODIC NONE                     ! System is not periodic
      POISSON_SOLVER WAVELET            ! Efficient solver for non-periodic system
    &END POISSON
    &SCF
      SCF_GUESS RESTART                 ! Initial density guess is a simple superposition of atomic charge densities
      EPS_SCF 1.0E-6                    ! Energy convergence criteria (atomic units)
      &OT                               ! Use orbital transformation method instead of brute-force diagonalization
        ALGORITHM IRAC                  ! OT algorithm
        MINIMIZER DIIS                  ! Minimization algorithm
        PRECONDITIONER FULL_ALL         ! Preconditioner for the minimizer
      &END OT
      MAX_SCF 20                        ! Maximum number of SCF steps before rebuilding preconditioner
      &OUTER_SCF ON
        EPS_SCF 1.0E-6                  ! Outer loop energy convergence criteria (atomic units)
        MAX_SCF 12                      ! Maximum number of preconditioner rebuilds before terminating
      &END OUTER_SCF
      &PRINT
        &RESTART
          BACKUP_COPIES 0               ! Avoid excessive printing of restart files
        &END RESTART
      &END PRINT
    &END SCF
    &XC
      &XC_FUNCTIONAL PBE                ! Exchange-correlation funtional
      &END XC_FUNCTIONAL
    &END XC
  &END DFT
  &SUBSYS
    &CELL
      PERIODIC NONE                     ! System is not periodic
      ABC 15.0 15.0 15.0                ! Simulation cell dimensions in angstroms
    &END CELL
    &TOPOLOGY                           ! Specify input coordinate format and filename
      COORD_FILE_FORMAT XYZ
      COORD_FILE_NAME enol-neb-pos-Replica_nr_5-1.xyz
    &END TOPOLOGY
    &KIND H                             ! Basis and pseudopotential for hydrogen
      BASIS_SET DZVP-MOLOPT-SR-GTH
      POTENTIAL GTH-PBE-q1
    &END KIND
    &KIND C                             ! Basis and pseudopotential for carbon
      BASIS_SET DZVP-MOLOPT-SR-GTH
      POTENTIAL GTH-PBE-q4
    &END KIND
    &KIND O                             ! Basis and pseudopotential for oxygen
      BASIS_SET DZVP-MOLOPT-SR-GTH
      POTENTIAL GTH-PBE-q6
    &END KIND
  &END SUBSYS
&END FORCE_EVAL

&VIBRATIONAL_ANALYSIS
  NPROC_REP 10
&END VIBRATIONAL_ANALYSIS

Writing nma.inp


In [7]:
%%writefile job.sh
#!/bin/bash
#SBATCH --time=00:15:00
#SBATCH --partition=test
#SBATCH --nodes=2
#SBATCH --ntasks-per-node=40
#SBATCH --account=project_2006657

module purge
module load intel-oneapi-compilers-classic/2021.6.0
module load intel-oneapi-mpi/2021.6.0
module load cp2k/2024.1

srun cp2k.psmp nma.inp

Writing job.sh


In [8]:
%%bash
unset ${!SLURM@}; sbatch job.sh

Submitted batch job 21072734


## Step 2.2: Analysis

Inspect the produced `.mol` file:

In [60]:
%%bash
cat enol-nma-VIBRATIONS-1.mol

 [Molden Format]
 [Atoms] AU
 C        1       6      13.129020      16.425523      14.135071
 C        2       6      15.409006      15.036031      13.951747
 C        3       6      15.416695      12.175004      13.745128
 C        4       6      12.959324      11.006297      14.767686
 C        5       6      10.651257      12.315491      13.592426
 C        6       6      10.589407      15.136901      14.284737
 H        7       1      17.060936      11.398597      14.753686
 H        8       1      12.920893       8.967934      14.366480
 H        9       1       8.890040      11.404331      14.211684
 H       10       1       9.253646      16.206989      13.104124
 O       11       8      13.168899      18.870227      14.260370
 C       12       6      17.645141      16.456009      13.994904
 O       13       8      17.689630      18.908759      14.158617
 H       14       1      19.484969      15.503313      13.911106
 H       15       1      15.472065      19.339307      14.216

* Check the lines under `[FREQ]`. Is there only one imaginary (negative) frequency?
* Scroll to the end of the file. How many vibrational modes are there in total? Does the number make sense?
    * Hint: think about degrees of freedom.

### Extra: Visualize the imaginary vibrational mode

[Molden](https://docs.csc.fi/apps/molden/) can be used to visualize `.mol` files. However, in a Jupyter Notebook we need to use other tools such as py3Dmol. To this end, create a new `.xyz` file that contains both the atomic and normal mode coordinates:

In [72]:
%%bash
tail -21 enol-neb-pos-Replica_nr_5-1.xyz > coord.tmp
grep -A20 FR-NORM-COORD enol-nma-VIBRATIONS-1.mol > vib.tmp
paste coord.tmp vib.tmp > vib.xyz

Then, visualize the mode using py3Dmol:

In [81]:
import py3Dmol

with open("vib.xyz", "r") as f:
    vib = f.read()

view = py3Dmol.view(height=400, width=400)
view.addModel(vib, "xyz", {"vibrate": {"frames": 10, "amplitude": 0.5}})
view.setStyle({"sphere": {"scale": 0.30}, "stick": {"radius": 0.25}})
view.animate({"loop": "backAndForth"})
view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

**Continue the hands-on with [Part 3](../Part-3-Hybrid/Part-3-Hybrid.ipynb).**